## 1. Connection with Azure and Spark (DELETE KEYS!)


In [0]:
storage_account_key = "wgbe0Fzs4W3dPNc35dp//uumz+SPDXVLLGu0mNaxTs2VLHCCPnD7u79PYt4mKeSFboqMRnZ+s+ez+ASty+k+sQ=="
storage_account_name = "factoredatathon2024"
container_name = "gold"

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",
    f"{storage_account_key}"
)

file_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/weightedAvgGoldsteinToneGold.csv"
df = spark.read.format("csv").option("header", "false").load(file_path)

## 2.Library Imports:


In [0]:
%pip install xgboost

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA

     

In [0]:
column_names = ["DATE", "Country", "GoldsteinScaleWA","ToneWA"]
df = df.toDF(*column_names)
events = df.toPandas()

display(events)

In [0]:
events.value_counts('Country')

## First Study Case
    

In [0]:
country_selected = 'CH' # Replace 'CH' with the name of the country you want to filter
events_filtered = events[events.Country == country_selected].copy()

In [0]:
events_filtered['DATE'] = pd.to_datetime(events_filtered['DATE'])
events_filtered.set_index('DATE', inplace=True)

In [0]:
#temp variables
#events_filtered.DATE = pd.DatetimeIndex(events_filtered.DATE)
#events_filtered['DATE'].dtype


In [0]:
events_filtered.head()

In [0]:
#events_filtered.index=pd.DatetimeIndex(events_filtered['DATE'], freq='D')
#events_filtered.drop(columns=['DATE'], inplace=True)

In [0]:
events_filtered.head()

In [0]:
events_filtered['GoldsteinScaleWA'] = events_filtered['GoldsteinScaleWA'].astype(float)

In [0]:
events_filtered['ToneWA'] = events_filtered['ToneWA'].astype(float)

In [0]:
events_filtered.dtypes

In [0]:
events_filtered['day'] = events_filtered.index.day
events_filtered['week'] = events_filtered.index.week
events_filtered['month'] = events_filtered.index.month
events_filtered['year'] = events_filtered.index.year
events_filtered['day_of_week'] = events_filtered.index.dayofweek

In [0]:
events_filtered.sort_index(ascending=True, inplace=True)
events_filtered.head()

In [0]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 10)
events_filtered.plot(kind = "line", y = ['GoldsteinScaleWA']);

In [0]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 10)
events_filtered.plot(kind = "line", y = ['ToneWA'], color='blue');

In [0]:
# lag features 
events_filtered['GoldsteinScaleWA_lag1'] = events_filtered['GoldsteinScaleWA'].shift(1)
events_filtered['GoldsteinScaleWA_lag7'] = events_filtered['GoldsteinScaleWA'].shift(7)
events_filtered['GoldsteinScaleWA_lag30'] = events_filtered['GoldsteinScaleWA'].shift(30)
events_filtered['ToneWA_lag1'] = events_filtered['ToneWA'].shift(1)

In [0]:
# movil windows features 
events_filtered['GoldsteinScaleWA_roll7'] = events_filtered['GoldsteinScaleWA'].rolling(window=7).mean()
events_filtered['ToneWA_roll7'] = events_filtered['ToneWA'].rolling(window=7).mean()

In [0]:
events_filtered.dropna(inplace=True)

In [0]:

X = events_filtered[['day', 'week', 'month', 'year', 'day_of_week', 'GoldsteinScaleWA_lag1','GoldsteinScaleWA_lag7','GoldsteinScaleWA_lag30', 'ToneWA_lag1','GoldsteinScaleWA_roll7', 'ToneWA_roll7']]
y = events_filtered['GoldsteinScaleWA']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

time_model = RandomForestRegressor(n_estimators=100, random_state=42)
time_model.fit(X_train, y_train)

In [0]:
y_pred = time_model.predict(X_test)

In [0]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

In [0]:
y_test_pred = time_model.predict(X_test)

# Create the combined plot
plt.figure(figsize=(14, 7))

# Plot the real values of GoldsteinScale in the training set
plt.plot(y_train.index, y_train, label='Real Value - Train Set', color='blue')

# Plot the predictions of GoldsteinScale in the test set
plt.plot(y_test.index, y_test_pred, label='Prediction - Test Set', color='red', linestyle='--')

# Plot settings
plt.title('GoldsteinScale Trend: Real Values (Train) and Predictions (Test)')
plt.xlabel('Date')
plt.ylabel('GoldsteinScale')
plt.ylim([-5, 5])
plt.legend()
plt.grid(True)
plt.show()


In [0]:
# Create the combined plot
plt.figure(figsize=(14, 7))

# Plot the real values of GoldsteinScale in the training set
plt.plot(y_train.index, y_train, label='Real Value - Train Set', color='blue')

# Plot the predictions of GoldsteinScale in the test set
plt.plot(y_test.index, y_test_pred, label='Prediction - Test Set', color='red', linestyle='--')

# Plot the real values of GoldsteinScale in the test set
plt.plot(y_test.index, y_test, label='Real Value - Test Set', color='green')

# Plot settings
plt.title('GoldsteinScale Trend: Real Values (Train), Predictions (Test), and Real Values (Test)')
plt.xlabel('Date')
plt.ylabel('GoldsteinScale')
plt.ylim([-5, 5])
plt.xlim([pd.Timestamp('2024-05-01'), pd.Timestamp('2024-08-10')])
plt.legend()
plt.grid(True)
plt.show()


## USA Test:

In [0]:
country_selected = 'US'  # Replace 'CH' with the name of the country you want to filter
events_filtered = events[events.Country == country_selected].copy()

In [0]:
events_filtered['DATE'] = pd.to_datetime(events_filtered['DATE'])
events_filtered.set_index('DATE', inplace=True)

In [0]:
events_filtered['GoldsteinScaleWA'] = events_filtered['GoldsteinScaleWA'].astype(float)

In [0]:
events_filtered['ToneWA'] = events_filtered['ToneWA'].astype(float)

In [0]:
events_filtered['day'] = events_filtered.index.day
events_filtered['week'] = events_filtered.index.week
events_filtered['month'] = events_filtered.index.month
events_filtered['year'] = events_filtered.index.year
events_filtered['day_of_week'] = events_filtered.index.dayofweek

In [0]:
events_filtered['day_sin'] = np.sin(2 * np.pi * events_filtered['day'] / 31)
events_filtered['day_cos'] = np.cos(2 * np.pi * events_filtered['day'] / 31)
events_filtered['week_sin'] = np.sin(2 * np.pi * events_filtered['week'] / 52)
events_filtered['week_cos'] = np.cos(2 * np.pi * events_filtered['week'] / 52)
events_filtered['month_sin'] = np.sin(2 * np.pi * events_filtered['month'] / 12)
events_filtered['month_cos'] = np.cos(2 * np.pi * events_filtered['month'] / 12)
cycle_length = 10
events_filtered['year_sin'] = np.sin(2 * np.pi * events_filtered['year'] / cycle_length)
events_filtered['year_cos'] = np.cos(2 * np.pi * events_filtered['year'] / cycle_length)
events_filtered['day_of_week_sin'] = np.sin(2 * np.pi * events_filtered['day_of_week'] / 7)
events_filtered['day_of_week_cos'] = np.cos(2 * np.pi * events_filtered['day_of_week'] / 7)


In [0]:
events_filtered.sort_index(ascending=True, inplace=True)
events_filtered.head()

In [0]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 10)
events_filtered.plot(kind = "line", y = ['GoldsteinScaleWA']);

In [0]:
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 10)
events_filtered.plot(kind = "line", y = ['ToneWA'], color='blue');

In [0]:
# lag features
events_filtered['GoldsteinScaleWA_lag1'] = events_filtered['GoldsteinScaleWA'].shift(1)
events_filtered['GoldsteinScaleWA_lag7'] = events_filtered['GoldsteinScaleWA'].shift(7)
events_filtered['GoldsteinScaleWA_lag30'] = events_filtered['GoldsteinScaleWA'].shift(30)
events_filtered['ToneWA_lag1'] = events_filtered['ToneWA'].shift(1)

In [0]:
# movil windows features
events_filtered['GoldsteinScaleWA_roll7'] = events_filtered['GoldsteinScaleWA'].rolling(window=7).mean()
events_filtered['ToneWA_roll7'] = events_filtered['ToneWA'].rolling(window=7).mean()

In [0]:
events_filtered.dropna(inplace=True)

In [0]:

#X = events_filtered[['day', 'week', 'month', 'year', 'day_of_week', 'GoldsteinScaleWA_lag1','GoldsteinScaleWA_lag7','GoldsteinScaleWA_lag30', 'ToneWA_lag1','GoldsteinScaleWA_roll7', 'ToneWA_roll7']]

X = events_filtered[['day_sin', 'day_cos', 'week_sin', 'week_cos', 'month_sin', 'month_cos', 
                     'year_sin', 'year_cos', 'day_of_week_sin', 'day_of_week_cos', 
                     'GoldsteinScaleWA_lag1', 'GoldsteinScaleWA_lag7', 'GoldsteinScaleWA_lag30', 
                     'ToneWA_lag1', 'GoldsteinScaleWA_roll7', 'ToneWA_roll7']]

y = events_filtered['GoldsteinScaleWA']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

time_model = RandomForestRegressor(n_estimators=100, random_state=42)
time_model.fit(X_train, y_train)

In [0]:
y_pred = time_model.predict(X_test)

In [0]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

In [0]:
y_test_pred = time_model.predict(X_test)

# Create the combined plot
plt.figure(figsize=(14, 7))

# Plot the actual GoldsteinScale values in the training set
plt.plot(y_train.index, y_train, label='Actual Value - Train Set', color='blue')

# Plot the GoldsteinScale predictions in the test set
plt.plot(y_test.index, y_test_pred, label='Prediction - Test Set', color='red', linestyle='--')

# Plot settings
plt.title('GoldsteinScale Trend: Actual Values (Train) and Predictions (Test)')
plt.xlabel('Date')
plt.ylabel('GoldsteinScale')
plt.ylim([-5, 5])
plt.legend()
plt.grid(True)
plt.show()


In [0]:
# <<<<<<< Updated upstream
# train = pd.DataFrame({
#     'y_train.index': y_train.index,
#     'y_train': y_train.values
# })

# test = pd.DataFrame({
#     'y_test.index': y_test.index,
#     'y_pred': y_test_pred
# })

# real = pd.DataFrame({
#     'y_test.index': y_test.index,
#     'y_real': y_test
# })
# y_test.index, y_test

# # Save DataFrame to a CSV file in Databricks
# csv_path = '/Workspace/Shared/real.csv'  # The /dbfs/ prefix is used to write files to DBFS

# real.to_csv(csv_path, index=False)

# # Verify that the file is saved correctly (optional)
# dbutils.fs.ls('dbfs:/Workspace/Shared/')
#=======
y_test_pred = time_model.predict(X_test)

# Create the combined plot
plt.figure(figsize=(14, 7))

# Plot the actual GoldsteinScale values in the training set
plt.plot(y_train.index, y_train, label='Actual Value - Train Set', color='blue')

# Plot the GoldsteinScale predictions in the test set
plt.plot(y_test.index, y_test_pred, label='Prediction - Test Set', color='red', linestyle='--')

# Plot the actual GoldsteinScale values in the test set
plt.plot(y_test.index, y_test, label='Actual Value - Test Set', color='green')

# Plot settings
plt.title('GoldsteinScale Trend: Actual Values (Train), Predictions (Test), and Actual Values (Test)')
plt.xlabel('Date')
plt.ylabel('GoldsteinScale')
plt.ylim([-5, 5])
plt.xlim([pd.Timestamp('2024-06-01'), pd.Timestamp('2024-08-10')])
plt.legend()
plt.grid(True)
plt.show()



### Hyperparams Optimization:


In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:

#X = events_filtered[['day', 'week', 'month', 'year', 'day_of_week', 'GoldsteinScaleWA_lag1','GoldsteinScaleWA_lag7','GoldsteinScaleWA_lag30', 'ToneWA_lag1','GoldsteinScaleWA_roll7', 'ToneWA_roll7']]
X = events_filtered[['day_sin', 'day_cos', 'week_sin', 'week_cos', 'month_sin', 'month_cos', 
                     'year_sin', 'year_cos', 'day_of_week_sin', 'day_of_week_cos', 
                     'GoldsteinScaleWA_lag1', 'GoldsteinScaleWA_lag7', 'GoldsteinScaleWA_lag30', 
                     'ToneWA_lag1', 'GoldsteinScaleWA_roll7', 'ToneWA_roll7']]

y = events_filtered['GoldsteinScaleWA']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Define the model
rf = RandomForestRegressor()

# Define the hyperparameter grid to test
param_grid = {
    'n_estimators': [150,200,250,300,350],
    'max_depth': [None, 1, 2, 3, 4, 5, 10],
    'min_samples_split': [10, 20, 30],
    'min_samples_leaf': [4, 5, 6, 7],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Configure GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Train GridSearchCV
grid_search.fit(X_train, y_train)

# Show the best parameters found
print("Best parameters found:")
print(grid_search.best_params_)

# Evaluate the model with the best parameters
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error with the best parameters: {mse}")


In [0]:
best_rf = grid_search.best_estimator_
best_rf

In [0]:

# Make predictions
y_pred = best_rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error with the best parameters: {mse}")

rmse = mean_squared_error(y_test, y_pred, squared=False)  # squared=False to get the square root
print(f"Root Mean Squared Error (RMSE): {rmse}")
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score (R-squared): {r2}")

# Create the combined plot
plt.figure(figsize=(14, 7))

# Plot the real values of GoldsteinScale in the training set
plt.plot(y_train.index, y_train, label='Real Value - Train Set', color='blue')

# Plot the predictions of GoldsteinScale in the test set
plt.plot(y_test.index, y_test_pred, label='Prediction - Test Set', color='red', linestyle='--')

# Plot the real values of GoldsteinScale in the test set
plt.plot(y_test.index, y_test, label='Real Value - Test Set', color='green')

# Plot settings
plt.title('GoldsteinScale Trend: Real Values (Train), Predictions (Test), and Real Values (Test)')
plt.xlabel('Date')
plt.ylabel('GoldsteinScale')
plt.ylim([-5, 5])
plt.xlim([pd.Timestamp('2024-06-01'), pd.Timestamp('2024-08-10')]) 
plt.legend()
plt.grid(True)
plt.show()


In [0]:
#Mejores parámetros encontrados:
#{'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 300}
#Mean Squared Error con los mejores parámetros: 0.12350352406916495

#Mejores parámetros encontrados:
#{'max_depth': 5, 'max_features': 'auto', 'min_samples_leaf': 5, 'min_samples_split': 20, 'n_estimators': 200}
#Mean Squared Error con los mejores parámetros: 0.11963690587393314

#Mejores parámetros encontrados:
#{'max_depth': 3, 'max_features': 'auto', 'min_samples_leaf': 4, 'min_samples_split': 20, 'n_estimators': 300}
#Mean Squared Error con los mejores parámetros: 0.11862474667284435

In [0]:

#X = events_filtered[['day', 'week', 'month', 'year', 'day_of_week', 'GoldsteinScaleWA_lag1','GoldsteinScaleWA_lag7',# X = events_filtered[['day', 'week', 'month', 'year', 'day_of_week', 'GoldsteinScaleWA_lag1','GoldsteinScaleWA_lag7','GoldsteinScaleWA_lag30', 'ToneWA_lag1','GoldsteinScaleWA_roll7', 'ToneWA_roll7']]
X = events_filtered[['day_sin', 'day_cos', 'week_sin', 'week_cos', 'month_sin', 'month_cos', 
                     'year_sin', 'year_cos', 'day_of_week_sin', 'day_of_week_cos', 
                     'GoldsteinScaleWA_lag1', 'GoldsteinScaleWA_lag7', 'GoldsteinScaleWA_lag30', 
                     'ToneWA_lag1', 'GoldsteinScaleWA_roll7', 'ToneWA_roll7']]

y = events_filtered['GoldsteinScaleWA']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Define the model
rf = RandomForestRegressor()

# Define the hyperparameter grid to test
param_grid = {
    'n_estimators': [150, 200, 250, 300, 350],
    'max_depth': [None, 1, 2, 3, 4, 5, 10],
    'min_samples_split': [10, 20, 30],
    'min_samples_leaf': [4, 5, 6, 7],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')

# Train GridSearchCV
grid_search.fit(X_train, y_train)

# Display the best parameters found
print("Best parameters found:")
print(grid_search.best_params_)

# Evaluate the model with the best parameters
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error with the best parameters: {mae}")


In [0]:
# Make predictions
y_pred = best_rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error with the best parameters: {mse}")

rmse = mean_squared_error(y_test, y_pred, squared=False)  # squared=False to obtain the square root
print(f"Root Mean Squared Error (RMSE): {rmse}")
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score (R-squared): {r2}")

# Create the combined plot
plt.figure(figsize=(14, 7))

# Plot the actual values of GoldsteinScale in the training set
plt.plot(y_train.index, y_train, label='Actual Value - Train Set', color='blue')

# Plot the predictions of GoldsteinScale in the test set
plt.plot(y_test.index, y_test_pred, label='Prediction - Test Set', color='red', linestyle='--')

# Plot the actual values of GoldsteinScale in the test set
plt.plot(y_test.index, y_test, label='Actual Value - Test Set', color='green')

# Plot settings
plt.title('GoldsteinScale Trend: Actual Values (Train), Predictions (Test), and Actual Values (Test)')
plt.xlabel('Date')
plt.ylabel('GoldsteinScale')
plt.ylim([-5, 5])
plt.xlim([pd.Timestamp('2024-06-01'), pd.Timestamp('2024-08-10')]) 
plt.legend()
plt.grid(True)
plt.show()


##XGBoost + USA


In [0]:
# Define the model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Define the hyperparameter grid to test
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3]
}

# Configure GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')

# Train GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and the best model
best_xgb = grid_search.best_estimator_

# Make predictions
y_pred = best_xgb.predict(X_test)

# Calculate the metrics
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display the metrics
print(f"Best parameters: {grid_search.best_params_}")
print(f"Mean Squared Error with the best parameters: {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R-squared (R^2): {r2}")




In [0]:
# Show metrics
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error con los mejores parámetros: {mse}")

In [0]:
# Create the combined plot
plt.figure(figsize=(14, 7))

# Plot the actual values of GoldsteinScale in the training set
plt.plot(y_train.index, y_train, label='Actual Value - Train Set', color='blue')

# Plot the predictions of GoldsteinScale in the test set
plt.plot(y_test.index, y_test_pred, label='Prediction - Test Set', color='red', linestyle='--')

# Plot the actual values of GoldsteinScale in the test set
plt.plot(y_test.index, y_test, label='Actual Value - Test Set', color='green')

# Plot settings
plt.title('GoldsteinScale Trend: Actual Values (Train), Predictions (Test), and Actual Values (Test)')
plt.xlabel('Date')
plt.ylabel('GoldsteinScale')
plt.ylim([-5, 5])
plt.xlim([pd.Timestamp('2024-06-01'), pd.Timestamp('2024-08-10')]) 
plt.legend()
plt.grid(True)
plt.show()


## ARIMA + USA


In [0]:

events_filtered['GoldsteinLog'] = np.log(events_filtered.GoldsteinScaleWA)
events_filtered["GoldsteinLogShift1"] = events_filtered.GoldsteinLog.shift()
events_filtered["GoldsteinLogDiff"] = events_filtered.GoldsteinLog - events_filtered.GoldsteinLogShift1
ts = events_filtered.GoldsteinLog
ts_diff = events_filtered.GoldsteinLogDiff
ts_diff.dropna(inplace = True)

In [0]:
from statsmodels.tsa.stattools import acf, pacf

In [0]:
lag_acf = acf(ts_diff, nlags=20)
lag_acf

In [0]:
ACF = pd.Series(lag_acf)
ACF.plot(kind = "bar")

In [0]:
lag_pacf = pacf(ts_diff, nlags=20, method='ols');

In [0]:
PACF = pd.Series(lag_pacf)
PACF.plot(kind = "bar");

In [0]:
# Let's see which parameters are significantly different from zero

#Plot ACF: 
plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
plt.title('Autocorrelation Function')

#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts)),linestyle='--',color='gray')
plt.title('Partial Autocorrelation Function')
plt.tight_layout()

In [0]:
from statsmodels.tsa.arima.model import ARIMA
ts.head()

In [0]:
# Instantiate the model with parameters (p=1, d=0, q=1) based on the ACF and PACF analysis
# In this case, d=0 because we are working directly with the differences
model_AR1MA = ARIMA(ts_diff, order=(1,0,1))

# Fit the model
results_ARIMA = model_AR1MA.fit()
results_ARIMA.fittedvalues.head()


In [0]:
print(results_ARIMA.summary())

In [0]:
ts_diff.plot()
results_ARIMA.fittedvalues.plot();

In [0]:
ts_diff.sum()

In [0]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
predictions_ARIMA_diff.tail()


In [0]:
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_diff_cumsum.tail()

In [0]:
ts.iloc[0]

In [0]:
predictions_ARIMA_log = pd.Series(ts.iloc[0], index=ts.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log.tail()

In [0]:
events_filtered['GoldsteinPredicted'] = np.exp(predictions_ARIMA_log)

In [0]:
rmse = mean_squared_error(events_filtered.GoldsteinPredicted, events_filtered.GoldsteinScaleWA, squared=False)
rmse
